In [ ]:
import networkx as nx
from langchain_ollama.llms import OllamaLLM
from pydantic import BaseModel
from typing import List
import json

In [70]:
# Define your schema
class Entity(BaseModel):
    name: str
    type: str  # e.g., "Chemical", "Process"

class Relation(BaseModel):
    source: str
    target: str
    type: str  # e.g., "REACTS_WITH", "IS_PART_OF"

# Ollama LLM with JSON mode
llm = OllamaLLM(model="gemma3:1b", temperature=0)

def get_llm(model: str = "gemma3:1b") -> OllamaLLM:
    """Get the LLM instance."""
    return OllamaLLM(model=model, temperature=0)



In [71]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [72]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Neo4j driver
neo4j_driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(
        NEO4J_USERNAME,
        NEO4J_PASSWORD
    )
)

In [73]:
def store_kg_deprecated(entities: List[Entity], relations: List[Relation]):
    print("Storing kg to neo4j")
    with neo4j_driver.session() as session:
        # Create entities (nodes)
        for entity in entities:
            session.run(
                """
                MERGE (e:Entity {name: $name})
                SET e.type = $type
                """,
                name=entity.name,
                type=entity.type
            )
        
        # Create relations
        for rel in relations:
            session.run(
                """
                MATCH (a:Entity {name: $source})
                MATCH (b:Entity {name: $target})
                MERGE (a)-[r:RELATION {type: $type}]->(b)
                """,
                source=rel.source,
                target=rel.target,
                type=rel.type
            )

In [74]:
def store_kg(entities: List[Entity], relations: List[Relation]):
    """
    Store knowledge graph in Neo4j with proper labels and relationships
    """
    print("Storing knowledge graph to Neo4j")
    with neo4j_driver.session() as session:
        # Create entities (nodes) with their specific types as labels
        for entity in entities:
            if not entity.type:
                entity.type = "Unknown"  # Default type if none provided
                
            # Clean entity name for Cypher
            clean_name = entity.name.replace("'", "''")
            
            # Use entity type as a label in addition to Entity
            session.run(
                f"""
                MERGE (e:Entity:{entity.type} {{name: $name}})
                """,
                name=clean_name
            )
        
        # Create relations with their specific types as relationship types
        for rel in relations:
            if not rel.type:
                continue  # Skip relations without types
                
            if not rel.source or not rel.target:
                continue  # Skip incomplete relations
                
            # Clean names for Cypher
            clean_source = rel.source.replace("'", "''")
            clean_target = rel.target.replace("'", "''")
            rel_type = rel.type.upper().replace(" ", "_")
            
            # Use the relation type as the relationship type
            session.run(
                f"""
                MATCH (a:Entity {{name: $source}})
                MATCH (b:Entity {{name: $target}})
                MERGE (a)-[r:{rel_type}]->(b)
                """,
                source=clean_source,
                target=clean_target
            )


In [75]:
def extract_kg_deprecated(text: str, model) -> tuple[List[Entity], List[Relation]]:
    prompt = f"""
    Extract entities and relationships from the text below. Return JSON with:
    - "entities": [{{"name": "...", "type": "..."}}]
    - "relations": [{{"source": "...", "target": "...", "type": "..."}}]

    Text: {text}
    """
    response = get_llm(model=model).invoke(prompt).strip()
    print("LLM Response:", response)  # Debug: show raw output
    try:
        data = json.loads(response)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON returned by LLM: {e}")
    entities = [Entity(**e) for e in data["entities"]]
    relations = [Relation(**r) for r in data["relations"]]
    return entities, relations


In [76]:
def extract_kg(text: str, model) -> tuple[List[Entity], List[Relation]]:
    """
    Extract entities and relationships from text with improved prompt and error handling
    """
    prompt = f"""
    Extract entities and relationships from the text below. 
    
    For each entity, provide:
    - A specific name (avoid generic terms)
    - A specific type (e.g., "Person", "Chemical", "Process", "Organization")
    
    For each relationship, provide:
    - A source entity name (must match an entity you identified)
    - A target entity name (must match an entity you identified)
    - A specific relationship type (e.g., "REACTS_WITH", "IS_PART_OF", "WORKS_FOR")
    
    Return valid JSON with:
    {{
      "entities": [
        {{"name": "specific entity name", "type": "specific entity type"}}
      ],
      "relations": [
        {{"source": "source entity name", "target": "target entity name", "type": "relationship type"}}
      ]
    }}
    
    Text: {text}
    """
    
    try:
        response = get_llm(model=model).invoke(prompt).strip()
        print("LLM Response:", response)  # Debug: show raw output
        
        # Extract JSON part if there's extra text
        if "```json" in response:
            json_part = response.split("```json")[1].split("```")[0].strip()
            response = json_part
        elif "```" in response:
            json_part = response.split("```")[1].split("```")[0].strip()
            response = json_part
            
        try:
            data = json.loads(response)
        except json.JSONDecodeError:
            # Try to fix common JSON issues
            fixed_response = response.replace("'", '"')
            fixed_response = re.sub(r'(\w+):', r'"\1":', fixed_response)
            data = json.loads(fixed_response)
            
        # Validate and clean the data
        entities = []
        for e in data.get("entities", []):
            if "name" in e and e["name"]:
                entity = Entity(
                    name=e["name"],
                    type=e.get("type", "Unknown")
                )
                entities.append(entity)
                
        relations = []
        for r in data.get("relations", []):
            if "source" in r and r["source"] and "target" in r and r["target"]:
                relation = Relation(
                    source=r["source"],
                    target=r["target"],
                    type=r.get("type", "RELATED_TO")
                )
                relations.append(relation)
                
        return entities, relations
        
    except Exception as e:
        print(f"Error extracting knowledge graph: {e}")
        return [], []


In [85]:
%pip install re 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [ ]:
import json
import re
from typing import List

def sanitize_label(label: str) -> str:
    return re.sub(r'\W+', '_', label.strip())

def store_kg(entities: List[Entity], relations: List[Relation]) -> None:
    """
    Store knowledge graph in Neo4j with proper labels and relationships
    """
    print("Storing knowledge graph to Neo4j")
    with neo4j_driver.session() as session:
        for entity in entities:
            if not entity.type:
                entity.type = "Unknown"
            label = sanitize_label(entity.type)
            clean_name = entity.name.replace("'", "''")

            cypher = f"MERGE (e:Entity:{label} {{name: $name}})"
            session.run(cypher, name=clean_name)

        for rel in relations:
            if not (rel.type and rel.source and rel.target):
                continue
            rel_type = sanitize_label(rel.type.upper())
            clean_source = rel.source.replace("'", "''")
            clean_target = rel.target.replace("'", "''")

            session.run(
                """
                MATCH (a:Entity {name: $source})
                MATCH (b:Entity {name: $target})
                CALL apoc.merge.relationship(a, $rel_type, {}, {}, b, {}) YIELD rel
                RETURN rel
                """,
                source=clean_source,
                target=clean_target,
                rel_type=rel_type
            )

def extract_kg(text: str, model) -> tuple[List[Entity], List[Relation]]:
    """
    Extract entities and relationships from text with improved prompt and error handling
    """
    prompt = f"""
    Extract entities and relationships from the text below. Entities and relationships should be pertained to the important  
    
    For each entity, provide:
    - A specific name (avoid generic terms)
    - A specific type (e.g., "Person", "Chemical", "Process", "Organization")
    
    For each relationship, provide:
    - A source entity name (must match an entity you identified)
    - A target entity name (must match an entity you identified)
    - A specific relationship type (e.g., "REACTS_WITH", "IS_PART_OF", "WORKS_FOR")
    
    Return valid JSON with:
    {{
      "entities": [
        {{"name": "specific entity name", "type": "specific entity type"}}
      ],
      "relations": [
        {{"source": "source entity name", "target": "target entity name", "type": "relationship type"}}
      ]
    }}
    
    Text: {text}
    """
    try:
        response = get_llm(model=model).invoke(prompt).strip()
        print("LLM Response:", response)

        if "```json" in response:
            response = response.split("```json")[1].split("```")[0].strip()
        elif "```" in response:
            response = response.split("```")[1].split("```")[0].strip()

        try:
            data = json.loads(response)
        except json.JSONDecodeError:
            response = response.replace("'", '"')
            response = re.sub(r'(\w+):', r'"\1":', response)
            data = json.loads(response)

        entities = [Entity(name=e["name"], type=e.get("type", "Unknown"))
                    for e in data.get("entities", []) if "name" in e and e["name"]]

        relations = [Relation(source=r["source"], target=r["target"], type=r.get("type", "RELATED_TO"))
                     for r in data.get("relations", [])
                     if "source" in r and r["source"] and "target" in r and r["target"]]

        return entities, relations

    except Exception as e:
        print(f"Error extracting knowledge graph: {e}")
        return [], []


In [87]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load PDF
loader = PyPDFLoader("books/ncert_class_x_science_chapter_1.pdf")  # Update path
documents = loader.load()

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust based on your needs
    chunk_overlap=200
)
text_chunks = text_splitter.split_documents(documents)  # Now `text_chunks` is defined

In [90]:
with neo4j_driver.session() as session:
  session.run("MATCH (n) DETACH DELETE n")

In [91]:
with neo4j_driver.session() as session:
  res = session.run("MATCH (n) RETURN n")
  for record in res:
    print(record)

In [88]:
# Extract and store knowledge graph from text chunks
for chunk in text_chunks[:5]:
    text = chunk.page_content
    try:
        entities, relations = extract_kg(text, model="nuextract:3.8b-q3_K_M")
        store_kg(entities, relations)
    except Exception as e:
        print(f"Error processing chunk: {e}")


LLM Response: {
  "entities": [
    {
      "name": "square6milk",
inate type: "Chemical"
    },
    {
      "name": "iron tawa/pan/nail",
      "type": "Iron"
    },
    {
      "name": "grapes",
      "type": "Fruit"
    },
    {
      "name": "food",
      "type": "Chemical"
    },
    {
      "name": "we",
      "type": "Person"
    }
  ],
  "relations": [
    {
      "source": "square6milk",
      "target": "",
      "type": ""
    },
    {
      "source": "iron tawa/pan/nail",
      "target": "",
      "type": ""
    },
    {
      "source": "grapes",
      "target": "",
      "type": ""
    },
    {
      "source": "food",
      "target": "",
      "type": ""
    },
    {
      "source": "we",
      "target": "",
      "type": ""
    }
  ]
}
<|end-output|>
Error extracting knowledge graph: Expecting property name enclosed in double quotes: line 5 column 1 (char 53)
Storing knowledge graph to Neo4j
LLM Response: {
  "entities": [],
  "relations": []
}
<|end-output|>
Error extract

In [79]:
from langchain_community.vectorstores import Neo4jVector
from langchain_huggingface import HuggingFaceEmbeddings

embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Store document chunks with embeddings
vector_store = Neo4jVector.from_documents(
    documents=text_chunks,  # Split text from Step 1
    embedding=embedder,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="custom_rag",
    node_label="Chunk"
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Answer using the context below. Always use the knowledge graph.
Context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

def custom_rag(question: str, model: str):
    # 1. Retrieve relevant chunks
    retrieved = vector_store.similarity_search(question, k=3)
    context = "\n".join([doc.page_content for doc in retrieved])
    
    # 2. Optionally query Neo4j for graph context
    with neo4j_driver.session() as session:
        graph_data = session.run(
            """
            MATCH (e:Entity)-[r]->(f:Entity)
            WHERE toLower(e.name) CONTAINS toLower($query)
            RETURN e.name, r.type, f.name
            """,
            parameters={"query": question}  # Proper parameter passing
        )
        kg_context = "\n".join([f"{row['e.name']} → {row['r.type']} → {row['f.name']}" for row in graph_data])
    
    # 3. Generate answer
    full_context = f"Text Context:\n{context}\n\nKnowledge Graph:\n{kg_context}"
    response = get_llm(model=model).invoke(prompt.format(context=full_context, question=question))
    return response

In [51]:
from IPython.display import display, Markdown

In [81]:
display(Markdown(custom_rag(question="What is a chemical reaction?", model="gemma3:1b")))

Here’s the answer based on the provided context:

**A chemical reaction is a process where two or more substances change into a new substance.**

Here’s a breakdown of how the context supports this:

*   **Change in Matter:** The context explicitly states that "Whenever a chemical change occurs, we have already learnt about physical and chemical changes of matter in our previous classes."
*   **Symbolic Representation:** The text mentions "Equations must always be balanced." This indicates that chemical reactions are represented symbolically, showing the reactants and products.
*   **Examples:** The examples provided (burning of magnesium, formation of water, burning of natural gas) illustrate that reactions involve the transformation of substances.



In [53]:
display(Markdown(custom_rag(question="What is a chemical reaction?", model="deepseek-r1:1.5b")))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: type)} {position: line: 4, column: 30, offset: 134} for query: '\n            MATCH (e:Entity)-[r]->(f:Entity)\n            WHERE toLower(e.name) CONTAINS toLower($query)\n            RETURN e.name, r.type, f.name\n            '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it

<think>
Okay, so I'm trying to figure out what a chemical reaction is based on the context provided. Let me start by reading through the text carefully.

The text talks about various situations where substances change when left in certain conditions. For example, square6milk left at room temperature during summers, an iron tawa (which I think is a pan or something) exposed to humid atmosphere, grapes getting fermented, food cooked, food digested in our body, respire. In all these cases, the nature and identity of the initial substance have changed.

They mention that whenever a chemical change occurs, we can say that a chemical reaction has taken place. So, I think a chemical reaction is when one or more substances are transformed into different substances through a chemical process.

Looking at what they learned about chemical equations: A complete chemical equation represents reactants, products, and their physical states symbolically. It's balanced so the number of atoms on both sides are equal. Equations must always be balanced because you can't have unequal numbers of atoms in a reaction.

They also talk about types of reactions like combination, decomposition, displacement, etc., but I'm focusing on what a chemical reaction is first.

In Activity 1.4, they did burning of magnesium ribbon and saw it form magnesium oxide. So that's a combination reaction because two elements combine to make one product.

Another example was the formation of water from hydrogen and oxygen gas. That's also a combination reaction since two gases combine to form a liquid or gas.

They mentioned exothermic reactions, which are when heat is released along with forming products. The burning of natural gas and respiration are examples given as exothermic because they produce heat.

So putting it all together, a chemical reaction is any process where one or more substances (reactants) are transformed into different substances (products) through a chemical change. This can involve physical changes too, like melting or boiling, but the key is that new substances form and the composition of matter changes.

I think I've got the gist from the text. The context defines what a chemical reaction is by showing examples with reactions happening in everyday situations.
</think>

A chemical reaction is a process where one or more substances (reactants) are transformed into different substances (products) through a chemical change, resulting in new combinations of matter and energy. This transformation can involve physical changes such as melting or boiling but fundamentally involves the creation of distinct new substances. Examples include burning magnesium to form magnesium oxide, which is a combination reaction, and forming water from hydrogen and oxygen gas, also a combination reaction. Chemical reactions are characterized by their ability to produce heat (exothermic) or require energy (endothermic), as seen in the burning of natural gas and respiration.

In [27]:
display(Markdown(custom_rag("How to balance a chemical equation in general? Give an example.")))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: type)} {position: line: 4, column: 30, offset: 134} for query: '\n            MATCH (e:Entity)-[r]->(f:Entity)\n            WHERE toLower(e.name) CONTAINS toLower($query)\n            RETURN e.name, r.type, f.name\n            '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it

Here's the breakdown of the balancing process based on the context provided:

**1. Balancing the Chemical Equation (Eq. (1.2))**

*   **Step I: Draw Boxes**
    *   Reactants (LHS): Mg + O2 → MgO
    *   Products (RHS): MgO + H2

*   **Step II: Count Atoms**
    *   Reactant (LHS):
        *   Fe: 1
        *   H: 2
        *   O: 1
    *   Product (RHS):
        *   Fe: 3
        *   H: 2
        *   O: 4

*   **Step III:  Equalize Atoms**
    *   Since the number of oxygen atoms (O) is different on both sides, we need to adjust the coefficient of O2.
    *   To balance the oxygen atoms, we need to multiply the O2 by 2.
    *   New equation: Mg + 2O2 → MgO + H2

*   **Step IV:  Balance Hydrogen Atoms**
    *   The number of hydrogen atoms on the RHS is 2, while the number on the LHS is 2.  Therefore, we need to multiply the H2 by 2.
    *   New equation: Mg + 2O2 → MgO + 2H2

*   **Step V:  Identify Unbalanced Element**
    *   Only iron (Fe) is left unbalanced.

**Final Balanced Equation:**

Mg + 2O2 → MgO + 2H2

**Answer:** The equation is balanced.

In [42]:
display(Markdown(custom_rag(question="Give some examples of chemical reactions.", model="deepseek-r1:1.5b")))


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: type)} {position: line: 4, column: 30, offset: 134} for query: '\n            MATCH (e:Entity)-[r]->(f:Entity)\n            WHERE toLower(e.name) CONTAINS toLower($query)\n            RETURN e.name, r.type, f.name\n            '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it

<think>
Okay, so I'm trying to figure out how to answer the question about giving some examples of chemical reactions using the provided context. Let me start by reading through the context carefully.

The context talks about what a complete chemical equation looks like and explains that equations must be balanced. It also defines different types of chemical reactions: combination, decomposition, exothermic, endothermic, displacement, etc. Then it gives some examples of these reactions in activities and provides knowledge about exothermic reactions with specific examples.

Now, the question is asking for some examples of chemical reactions using the context. I need to use the knowledge graph provided, which probably has a list or definitions of various chemical reactions. 

Looking at the context again, it mentions that whenever a chemical change occurs, we can say a reaction has taken place. It also gives several activities and examples, like burning magnesium ribbon, forming water from hydrogen and oxygen, etc.

I should think about different types of chemical reactions mentioned in the context and provide specific examples for each type. The context lists combination, decomposition, exothermic, endothermic, displacement, double displacement, etc., but I need to focus on giving some examples for each main category or perhaps a few key ones.

Let me list out the main categories from the context:

1. Combination reactions
2. Decomposition reactions
3. Exothermic reactions
4. Endothermic reactions
5. Displacement reactions
6. Double displacement reactions

I can provide one example for each of these, but maybe I should also think about other types or perhaps more specific examples.

For combination reactions: The activity mentions burning magnesium ribbon and forming magnesium oxide. That's a good example.

Decomposition reactions: Formation of water from hydrogen and oxygen is another example.

Exothermic reactions: Burning natural gas and respiration are given as examples, but I can think of others like the decomposition of ammonium chloride to produce ammonia and chlorine gas (though that might be more complex).

Endothermic reactions: The context doesn't give many examples beyond what's listed. Maybe something like digestion or photosynthesis? Wait, photosynthesis is a chemical reaction too.

Displacement reactions: Iron tawa/pan/nail left exposed to humid atmosphere—maybe that's a displacement of oxygen from water, but I'm not sure if that's the best example. Alternatively, maybe it's more about acid-base reactions?

Wait, perhaps I should think of different types and provide examples for each.

Let me try to structure this:

1. Combination: Magnesium ribbon + oxygen → magnesium oxide
2. Decomposition: 2H₂ + O₂ → 2H₂O
3. Exothermic: Burning methane (CH₄) → CO₂ + H₂O, or digestion of food.
4. Endothermic: Photosynthesis (light energy to produce glucose and oxygen)
5. Displacement: Iron in acid solution reacts with another metal to form a new compound.

Wait, but displacement reactions are more about one element displacing another from its compound. So maybe something like the reaction between zinc and hydrochloric acid, where Zn + HCl → ZnCl₂ + H₂.

But I'm not sure if that's considered a displacement reaction in this context. Maybe it's better to stick with the examples given in the context or think of others.

Alternatively, perhaps I can list more specific chemical reactions from different areas like combustion, acid-base, photosynthesis, etc.

Let me try again:

1. Combination: Rusting (iron + oxygen → iron oxide)
2. Decomposition: Fire burning wood into carbon and other gases
3. Exothermic: Combustion of fuels (e.g., propane)
4. Endothermic: Photosynthesis in plants
5. Displacement: Reaction between metals and acids, like Zn + HCl → ZnCl₂ + H₂

I think that covers a good range of examples from different areas. Each example is a chemical reaction as per the context provided.
</think>

Here are some examples of chemical reactions based on the context provided:

1. **Combination Reaction**: Rusting (iron + oxygen → iron oxide)
2. **Decomposition Reaction**: Fire burning wood into carbon and other gases
3. **Exothermic Reaction**: Combustion of fuels (e.g., propane)
4. **Endothermic Reaction**: Photosynthesis in plants
5. **Displacement Reaction**: Reaction between metals and acids, e.g., Zn + HCl → ZnCl₂ + H₂

Each example illustrates different types of chemical reactions as discussed in the context.

In [43]:
display(Markdown(custom_rag(question="Is this equation balanced?  Zn + HCl → ZnCl₂ + H₂ ? if not balance it please", model="deepseek-r1:1.5b")))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: type)} {position: line: 4, column: 30, offset: 134} for query: '\n            MATCH (e:Entity)-[r]->(f:Entity)\n            WHERE toLower(e.name) CONTAINS toLower($query)\n            RETURN e.name, r.type, f.name\n            '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it

<think>
Okay, so I need to figure out if the chemical equation Zn + HCl → ZnCl₂ + H₂ is balanced. Let me start by recalling what a balanced equation means. All atoms must be accounted for on both sides of the arrow.

First, let's list the elements involved: Zinc (Zn), Hydrogen (H), and Chlorine (Cl). 

On the reactant side:
- Zn has 1 atom.
- H has 2 atoms (since HCl is H + Cl).
- Cl has 1 atom.

On the product side:
- ZnCl₂ has 1 Zn and 2 Cl atoms.
- H₂ has 2 H atoms.

Now, let's count each element:

Zinc: Reactant side has 1, product side has 1. That matches.

Hydrogen: Reactant side has 2, product side has 2. Wait, that seems correct too.

Chlorine: Reactant side has 1, product side has 2. Hmm, that's a problem. We have one Cl on the reactant and two on the product.

So, to balance chlorine, I need to add another HCl molecule on the reactant side. That would give me 2 Cl atoms in total.

Let me write it out:

Zn + 2 HCl → ZnCl₂ + H₂

Now, let's check each element again:

Zinc: Reactant has 1, product has 1. Balanced.

Hydrogen: Reactant has 4 (from 2 HCl), product has 2. That doesn't balance yet.

Chlorine: Reactant has 2, product has 2. Balanced.

So now, hydrogen is off by a factor of 2. To fix that, I can add a coefficient of 2 in front of H₂ on the product side:

Zn + 2 HCl → ZnCl₂ + 2 H₂

Now, let's check all elements again:

Zinc: 1 vs 1. Balanced.

Hydrogen: Reactant has 4 (from 2 HCl), product has 4 (2 H₂). Balanced.

Chlorine: 2 vs 2. Balanced.

So the balanced equation is Zn + 2 HCl → ZnCl₂ + 2 H₂.
</think>

The given chemical equation, Zn + HCl → ZnCl₂ + H₂, is not balanced because the number of chlorine atoms on the reactant side (1) does not match those on the product side (2). To balance it:

1. Add a coefficient of 2 in front of HCl: Zn + 2 HCl → ZnCl₂ + H₂.
2. Check all elements:
   - Zinc: 1 vs 1.
   - Hydrogen: 4 vs 4.
   - Chlorine: 2 vs 2.

The balanced equation is:

Zn + 2 HCl → ZnCl₂ + 2 H₂

**Answer:** The balanced chemical equation is Zn + 2 HCl → ZnCl₂ + 2 H₂.

In [44]:
custom_rag(question="Is this equation balanced?  Zn + HCl → ZnCl₂ + H₂ ? if not balance it please", model="deepseek-r1:1.5b")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: type)} {position: line: 4, column: 30, offset: 134} for query: '\n            MATCH (e:Entity)-[r]->(f:Entity)\n            WHERE toLower(e.name) CONTAINS toLower($query)\n            RETURN e.name, r.type, f.name\n            '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it

"<think>\nOkay, so I need to figure out if the chemical equation Zn + HCl → ZnCl₂ + H₂ is balanced. If it's not balanced, I have to balance it using the steps provided in the text. Let me start by recalling what a balanced chemical equation means. It means that the number of atoms of each element is the same on both sides of the arrow.\n\nFirst, let me write down the given equation:\n\nZn + HCl → ZnCl₂ + H₂\n\nI'll list out the elements and their counts on both sides.\n\nOn the reactant side (left):\n- Zinc (Zn): 1 atom\n- Hydrogen (H): 1 atom from HCl\n- Chlorine (Cl): 1 atom from HCl\n\nOn the product side (right):\n- Zinc (Zn): 2 atoms in ZnCl₂\n- Hydrogen (H): 2 atoms in H₂\n- Chlorine (Cl): 1 atom each in ZnCl₂ and H₂, so total 2 atoms\n\nNow, let's count them:\n\nReactants:\n- Zn: 1\n- H: 1\n- Cl: 1\n\nProducts:\n- Zn: 2\n- H: 2\n- Cl: 2\n\nComparing both sides, I see that the number of zinc and chlorine atoms isn't balanced. Zinc is only 1 on the left but 2 on the right. Chlorin

In [89]:
display(Markdown(custom_rag(question="Explain decomposition of calcium carbonate", model="deepseek-r1:1.5b")))

<think>
Okay, so I need to explain the decomposition of calcium carbonate using the context provided. Let me start by recalling what I know about decomposition reactions. From the context, I see that decomposition reactions break down a single compound into simpler substances.

Looking at the examples given in the context: when calcium carbonate (CaCO3) is heated, it decomposes into calcium oxide (CaO) and carbon dioxide gas (CO2). The reaction is written as CaCO3(s) → CaO(s) + CO2(g). This makes sense because when you heat limestone or marble, it loses water and forms calcium hydroxide, which then reacts with carbon dioxide to produce calcium carbonate and water.

I should also mention the chemical formula for marble, which is CaCO3. That's important because it shows that marble is a form of calcium carbonate. The reaction is exothermic, meaning it releases heat, but I don't know if that's relevant here since the question is about decomposition itself.

Wait, in the context, there are other examples like slaked lime and ferrous sulphate reacting to produce iron(III) oxide. But for calcium carbonate, the key part is the heating reaction producing CaO and CO2 gas. I should make sure to include that in my explanation.

I think I have all the necessary information from the context. Now, putting it together: explain the decomposition of CaCO3 into CaO and CO2, mention the chemical formula for marble, and perhaps note how this is used in industries like cement manufacturing.
</think>

The decomposition reaction of calcium carbonate (CaCO3) involves heating to produce simpler substances. When calcium carbonate is heated, it breaks down into calcium oxide (CaO), a basic compound, and carbon dioxide gas (CO2). This can be represented as:

\[ \text{CaCO}_3(s) \rightarrow \text{CaO}(s) + \text{CO}_2(g) \]

The chemical formula for marble is CaCO3, which is an example of calcium carbonate. This reaction is crucial in industries like cement manufacturing, where it helps produce lime (calcium oxide), a key component in making cement. The process also results in a shiny finish due to the formation of calcium carbonate on the walls after whitewashing.